# Bayesian Logistic Regression
[Bayesian logistic regression](https://en.wikipedia.org/wiki/Logistic_regression#Bayesian) is the Bayesian counterpart to a common tool in machine learning, logistic regression. The goal of logistic regression is to predict a one or a zero for a given training item. An example might be predicting whether someone is sick or ill given their symptoms and personal information.

In our example, we'll be working to predict whether someone is likely to default with a synthetic dataset found in the `RDatasets` package. This dataset, `Defaults`, comes from R's [ISLR](https://cran.r-project.org/web/packages/ISLR/index.html) package and contains information on borrowers.

To start, let's import all the libraries we'll need.

In [ ]:
# Import Turing and Distributions.
using Turing, Distributions

# Import RDatasets.
using RDatasets

# Import MCMCChain, Plots, and StatPlots for visualizations and diagnostics.
using MCMCChain, Plots, StatPlots

# We need a logistic function, which is provided by StatsFuns.
using StatsFuns: logistic

# MLDataUtils provides a sample splitting tool that's very handy.
using MLDataUtils

## Data Cleaning & Set Up

Now we're going to import our dataset. The first six rows of the dataset are shown below so you capn get a good feel for what kind of data we have.

In [ ]:
# Import the "Default" dataset.
data = RDatasets.dataset("ISLR", "Default");

# Show the first six rows of the dataset.
head(data)

Most machine learning processes require some effort to tidy up the data, and this is no different. We need to convert the `Default` and `Student` columns, which say "Yes" or "No" into 1s and 0s. Afterwards, we'll get rid of the old words-based columns.

In [ ]:
# Create new rows, defualted to zero.
data[:DefaultNum] = 0.0
data[:StudentNum] = 0.0

for i in 1:length(data.Default)
    # If a row's "Default" or "Student" columns say "Yes",
    # set them to 1 in our new columns.
    data[:DefaultNum][i] = data.Default[i] == "Yes" ? 1.0 : 0.0
    data[:StudentNum][i] = data.Student[i] == "Yes" ? 1.0 : 0.0
end

# Delete the old columns which say "Yes" and "No".
delete!(data, :Default)
delete!(data, :Student)

# Show the first six rows of our edited dataset.
head(data)

After we've done that tidying, it's time to split our dataset into training and testing sets, and separate the labels from the data. We use `MLDataUtils.splitobs` to separate our data into two halves, `train` and `test`. You can use a higher percentage of splitting (or a lower one) by modifying the `at = 0.5` argument. We have set the splitting proportion to 50%, as Bayesian logistic regression tends to take a longer period of time with larger sample sizes than typicaly frequentist methods.

In [ ]:
# Split our dataset 70/30 into training/test sets.
train, test = MLDataUtils.splitobs(data, at = 0.5);

# Create our labels. These are the values we are trying to predict.
train_label = train[:DefaultNum]
test_label = test[:DefaultNum]

# Remove the columns that are not our predictors.
train = train[[:StudentNum, :Balance, :Income]]
test = test[[:StudentNum, :Balance, :Income]]

Our `train` and `test` matrices are still in the `DataFrame` format, which tends not to play too well with the kind of manipulations we're about to do, so we convert them into `Matrix` objects.

In [ ]:
# Convert the DataFrame objects to matrices.
train = Matrix(train);
test = Matrix(test);

This next part is critically important. We must rescale our variables so that they are centered around zero by subtracting each column by the mean and dividing it by the standard deviation. Without this step, Turing's sampler will have a hard time finding a place to start searching for parameter estimates.

In [ ]:
# Rescale our matrices.
train = (train .- mean(train, dims=1)) ./ std(train, dims=1)
test = (test .- mean(test, dims=1)) ./ std(test, dims=1)

## Model Declaration 
Finally, we can define our model.

`logistic regression` takes four arguments:

- `x` is our set of independent variables;
- `y` is the element we want to predict;
- `n` is the number of observations we have; and
- `σ²` is the standard deviation we want to assume for our priors.

Within the model, we create four coefficients (`intercept`, `student`, `balance`, and `income`) and assign a prior of normally distributed with means of zero and standard deviations of `σ²`. We want to find values of these four coefficients to predict any given `y`.

The `for` block creates a variable `v` which is the logistic function. We then observe the liklihood of calculating `v` given the actual label, `y[i]`.

In [ ]:
# Bayesian logistic regression (LR)
@model logistic_regression(x, y, n, σ²) = begin
    intercept ~ Normal(0, σ²)

    student ~ Normal(0, σ²)
    balance ~ Normal(0, σ²)
    income  ~ Normal(0, σ²)

    for i = 1:n
        v = logistic(intercept + student*x[i, 1] + balance*x[i,2] + income*x[i,3])
        y[i] ~ Bernoulli(v)
    end
end

## Sampling

Now we can run our sampler. This time we'll use [`HMC`](http://turing.ml/docs/library/#Turing.HMC) to sample from our posterior.

In [ ]:
# This is temporary while the reverse differentiation backend is being improved.
Turing.setadbackend(:forward_diff)

# Retrieve the number of observations.
n, _ = size(test)

# Sample using HMC.
chain = sample(logistic_regression(train, train_label, n, 1), HMC(1500, 0.05, 10))

We can use the `cornerplot` function from StatPlots to show the distributions of the various parameters of our logistic regression. 

In [ ]:
# The labels to use.
l = [:student, :balance, :income]

# Extract the parameters we want to plot.
w1 = chain[:student]
w2 = chain[:balance]
w3 = chain[:income]

# Show the corner plot.
cornerplot(hcat(w1, w2, w3), compact=true, labels = l)

Fortunately the corner plot appears to demonstrate unimodal distributions for each of our parameters, so it should be straightforward to take the means of each parameter's sampled values to estimate our model to make predictions.

## Making Predictions
How do we test how well the model actually predicts whether someone is likely to default? We need to build a prediction function that takes the `test` object we made earlier and runs it through the average parameter calculated during sampling.

The `prediction` function below takes a `Matrix` and a `Chain` object. It takes the mean of each parameter's sampled values and re-runs the logistic function using those mean values for every element in the test set.

In [ ]:
function prediction(x::Matrix, chain)
    # Pull the means from each parameter's sampled values in the chain.
    intercept = mean(chain[:intercept])
    student = mean(chain[:student])
    balance = mean(chain[:balance])
    income = mean(chain[:income])

    # Retrieve the number of rows.
    n, _ = size(x)

    # Generate a vector to store our predictions.
    v = Vector{Float64}(undef, n)

    # Calculate the logistic function for each element in the test set.
    for i in 1:n
        v[i] = logistic(intercept .+ student * x[i,1] + balance * x[i,2] + income * x[i,3])
    end
    return v
end

Let's see how we did! We run the test matrix through the prediction function, and compute the [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error) (MSE) for our prediction.

In [ ]:
# Make the predictions.
predictions = prediction(test, chain)

# Calculate MSE for our test set.
loss = sum((predictions - test_label).^2) / length(test_label)

Perhaps more important is to see what percentage of defaults we correctly predicted. The code below simply counts defaults and predictions and presents the results. The `threshold` variable sets the sensitivity of the predictions. For example, a threshold of 0.10 will predict a defualt value of 1 for any predicted value greater than 1.0 and no default if it is less than 0.10.

In [ ]:
defaults = 0
not_defaults = 0

predicted_defaults = 0
predicted_not_defaults = 0

threshold = 0.10

for i in 1:length(predictions)
    if test_label[i] == 1.0
        defaults += 1
        
        if predictions[i] >= threshold
            predicted_defaults +=  1
        end
    else 
        not_defaults += 1
        if predictions[i] <= threshold
            predicted_not_defaults +=  1
        end
    end
end

println("Defaults: $defaults
    Correct predictions: $num_predictions
    Percentage defaults correct $(predicted_defaults/defaults)")

println("Not defaults: $not_defaults
    Correct predictions: $predicted_not_defaults
    Percentage defaults correct $(predicted_not_defaults/not_defaults)")

The above shows that with a threshold of 0.10, we correctly predict a respectable portion of the defaults, and correctly identify most non-defaults. This is fairly sensitive to a choice of threshold, and you may wish to experiment with it.

This tutorial has demonstrated how to use Turing to perform Bayesian logistic regression. 